In [1]:
from SW import *
import numpy as np
from fxpmath import Fxp
import itertools
N=4
INT=4
DEC=4
MAC=2
S=4
#S=S*N*2
Th=8

Layers=int(log2(N))
Bits=DEC+INT
Size=N*Bits
SelCounts=int(N*Layers/2/MAC)
TotSize=N*Layers#N*L


In [2]:
RandomBinary(S,N,INT,DEC,Th)

[array(["32'h53_79_34_01", "32'h33_21_5C_3C", "32'h1D_65_58_17",
        "32'h67_46_2E_54"], dtype='<U15'),
 array(["32'h40_54_06_5C", "32'h77_0D_7F_75", "32'h6E_77_4C_10",
        "32'h7C_3A_3A_14"], dtype='<U15'),
 array([[5.1875+4.j    , 7.5625+5.25j  , 3.25  +0.375j , 0.0625+5.75j  ],
        [3.1875+7.4375j, 2.0625+0.8125j, 5.75  +7.9375j, 3.75  +7.3125j],
        [1.8125+6.875j , 6.3125+7.4375j, 5.5   +4.75j  , 1.4375+1.j    ],
        [6.4375+7.75j  , 4.375 +3.625j , 2.875 +3.625j , 5.25  +1.25j  ]])]

array([[0.39685455],
       [0.07769206],
       [0.07464769],
       [0.89634761],
       [0.22941009]])

In [3]:
Fxp(1,True,8,4).bin()

'00010000'

In [53]:
Fxp(np.exp(-2j*np.pi*n/k),True,8,4).bin()

TypeError: only size-1 arrays can be converted to Python scalars

In [24]:
MAC_Indexes(N,MAC).shape

(40, 8, 6)

In [25]:
out=np.zeros([SelCounts,int(TotSize/SelCounts),3],dtype=object)

t1=np.repeat(np.arange(MAC),2,axis=0)[None,:] 
out[:,:,0]=t1

t1=np.arange(2)[None,:]
t2=np.repeat(t1,TotSize/SelCounts/2,axis=0).reshape([1,-1])
out[:,:,1]=t2


k=np.array([2**i for i in range(1,int(log2(N)+1))])
n=np.array([list(range(i))*int(N/i) for i in k] )[:,0:int(N/2)]
k=k[:,None]
const=np.exp(-2j*2*np.pi*n/k)

ConstBinary=np.array([[ str(Bits)+"'b"+Fxp(i.real,True,Bits,DEC).bin(),
                        str(Bits)+"'b"+Fxp(i.imag,True,Bits,DEC).bin()] 
for i in const.ravel()]).reshape(-1,int(N/2),2)
t1=ConstBinary.reshape([SelCounts,-1])
out[:,:,2]=t1
out[:,:,2]

array([["8'b00010000", "8'b00000000", "8'b00010000", "8'b00000000"],
       ["8'b00010000", "8'b00000000", "8'b00010000", "8'b00000000"],
       ["8'b00010000", "8'b00000000", "8'b00010000", "8'b00000000"],
       ["8'b00010000", "8'b00000000", "8'b00010000", "8'b00000000"],
       ["8'b00010000", "8'b00000000", "8'b00010000", "8'b00000000"],
       ["8'b00010000", "8'b00000000", "8'b00010000", "8'b00000000"],
       ["8'b00010000", "8'b00000000", "8'b00010000", "8'b00000000"],
       ["8'b00010000", "8'b00000000", "8'b00010000", "8'b00000000"],
       ["8'b00010000", "8'b00000000", "8'b11110000", "8'b00000000"],
       ["8'b00010000", "8'b00000000", "8'b11110000", "8'b00000000"],
       ["8'b00010000", "8'b00000000", "8'b11110000", "8'b00000000"],
       ["8'b00010000", "8'b00000000", "8'b11110000", "8'b00000000"],
       ["8'b00010000", "8'b00000000", "8'b11110000", "8'b00000000"],
       ["8'b00010000", "8'b00000000", "8'b11110000", "8'b00000000"],
       ["8'b00010000", "8'b0000000

In [26]:
def MulConst(n,d,INT,DEC):

    c1=cos(2*pi*n/d)
    s1=-sin(2*pi*n/d)
    
    x = Fxp(c1, True, INT+DEC, DEC)      # signed=True, n_word=16, n_frac=8

    a1=x.bin(frac_dot=True)
    
    x = Fxp(s1, True, INT+DEC, DEC)      # signed=True, n_word=16, n_frac=8

    a2=x.bin(frac_dot=True)    
    return [a1,a2]



k=[[[i,2**j] for i in range(int(2**(j-1)))] for j in range(1,int(log2(N)+1))]

#L_n_d
C=[[MulConst(n=i[0], d=i[1],INT=INT,DEC=DEC) for l,i in enumerate(j)]   for j in k]

def m1(e):
    def m2(e):
        def m3(e):
            return e.replace('.','_')
        return list(map(m3,e))
    return list(map(m2,e))

C=list(map(m1,C))

def m(a):
    return a*int(N/(2*len(a)))
MulC=np.array(list(map(lambda a:a*int(N/(2*len(a))),C)))
MulC=np.expand_dims(MulC,2)
MulC.shape

(5, 16, 1, 2)

In [27]:
out=np.zeros([SelCounts,TotSize*2])

In [28]:
def MAC_Constants(N,INT,DEC):
    """
    N=> size of FFT
    INT=> number of integer bits
    DEC => number of decimal bits
    return => shape=(5, 16, 2) for: 5 layers , 16 Multiplier , (real , imag)
    """
    def MulConst(n,d,INT,DEC):

        c1=cos(2*pi*n/d)
        s1=-sin(2*pi*n/d)
        
        x = Fxp(c1, True, INT+DEC, DEC)      # signed=True, n_word=16, n_frac=8

        a1=x.bin(frac_dot=True)
        
        x = Fxp(s1, True, INT+DEC, DEC)      # signed=True, n_word=16, n_frac=8

        a2=x.bin(frac_dot=True)    
        return [a1,a2]

    
    
    k=[[[i,2**j] for i in range(int(2**(j-1)))] for j in range(1,int(log2(N)+1))]

    #L_n_d
    C=[[MulConst(n=i[0], d=i[1],INT=INT,DEC=DEC)+[f'{l}_{i[0]}_{i[1]}'] for l,i in enumerate(j)]   for j in k]

    def m1(e):
        def m2(e):
            def m3(e):
                return e.replace('.','_')
            return list(map(m3,e))
        return list(map(m2,e))

    C=list(map(m1,C))

    def m(a):
        return a*int(N/(2*len(a)))
    MulC=np.array(list(map(lambda a:a*int(N/(2*len(a))),C)))
    MulC=np.expand_dims(MulC,2)
    return MulC